# Preprocessing

In [3]:
# import libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
import random
from tensorflow.keras.preprocessing import image
import kerastuner as kt
from tensorflow.keras.models import load_model


/var/folders/tr/pkwjp9qn76v_83kx3jf06q6w0000gn/T/ipykernel_31512/3754220758.py:12: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [4]:
### Divide dataset into training and validation sets ###

# # Parameters
# source_folder = 'dogs-vs-cats/original'
# train_folder = 'dogs-vs-cats/train'
# test_folder = 'dogs-vs-cats/test'
# test_ratio = 0.2  # 20% for test, 80% for train

# # List all jpg files (customize for othfer extensions if needed)
# all_files = [f for f in os.listdir(source_folder) if f.lower().endswith('.jpg')]

# # Shuffle and split
# random.shuffle(all_files)
# split_index = int(len(all_files) * (1 - test_ratio))
# train_files = all_files[:split_index]
# test_files = all_files[split_index:]

# # Move files
# for f in train_files:
#     shutil.move(os.path.join(source_folder, f), os.path.join(train_folder, f))

# for f in test_files:
#     shutil.move(os.path.join(source_folder, f), os.path.join(test_folder, f))

# print(f"Moved {len(train_files)} files to train and {len(test_files)} to test.")


In [ ]:
# # Set your source and destination folders
# source_folder = 'dogs-vs-cats/train'
# destination_folder_dogs = 'dogs-vs-cats/train/dogs'
# destination_folder_cats = 'dogs-vs-cats/train/cats'


# # Loop through files in source folder
# for filename in os.listdir(source_folder):
#     # Check if filename starts with "dog" and ends with ".jpg"
#     if filename.lower().startswith('dog') and filename.lower().endswith('.jpg'):
#         src_path = os.path.join(source_folder, filename)
#         dst_path = os.path.join(destination_folder_dogs, filename)

#         # Copy the file to destination folder
#         shutil.copy2(src_path, dst_path)

#         # Delete the original file
#         os.remove(src_path)

#     elif filename.lower().startswith('cat') and filename.lower().endswith('.jpg'):
#         src_path = os.path.join(source_folder, filename)
#         dst_path = os.path.join(destination_folder_cats, filename)

#         # Copy the file to destination folder
#         shutil.copy2(src_path, dst_path)

#         # Delete the original file
#         os.remove(src_path)

# source_folder = 'dogs-vs-cats/test'
# destination_folder_dogs = 'dogs-vs-cats/test/dogs'
# destination_folder_cats = 'dogs-vs-cats/test/cats'


# # Loop through files in source folder
# for filename in os.listdir(source_folder):
#     # Check if filename starts with "dog" and ends with ".jpg"
#     if filename.lower().startswith('dog') and filename.lower().endswith('.jpg'):
#         src_path = os.path.join(source_folder, filename)
#         dst_path = os.path.join(destination_folder_dogs, filename)

#         # Copy the file to destination folder
#         shutil.copy2(src_path, dst_path)

#         # Delete the original file
#         os.remove(src_path)

#     elif filename.lower().startswith('cat') and filename.lower().endswith('.jpg'):
#         src_path = os.path.join(source_folder, filename)
#         dst_path = os.path.join(destination_folder_cats, filename)

#         # Copy the file to destination folder
#         shutil.copy2(src_path, dst_path)

#         # Delete the original file
#         os.remove(src_path)

In [6]:
train_dir = 'dogs-vs-cats/train'
test_dir = 'dogs-vs-cats/test'

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [7]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

/Users/suzukikenta/.pyenv/versions/3.12.9/envs/lewagon/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# Training and evaluation

In [ ]:
def build_model(hp):
    model = Sequential()

    # First Conv Block
    model.add(Conv2D(
        filters=hp.Int('conv1_filters', min_value=32, max_value=128, step=32),
        kernel_size=3,
        activation='relu',
        padding='same',
        input_shape=(150,150,3)
    ))
    model.add(MaxPooling2D(pool_size=2))

    # Second Conv Block
    model.add(Conv2D(
        filters=hp.Int('conv2_filters', min_value=32, max_value=128, step=32),
        kernel_size=3,
        activation='relu',
        padding='same'
    ))
    model.add(MaxPooling2D(pool_size=2))

    # Third Conv Block
    model.add(Conv2D(
        filters=hp.Int('conv3_filters', min_value=32, max_value=128, step=32),
        kernel_size=3,
        activation='relu',
        padding='same'
    ))
    model.add(MaxPooling2D(pool_size=2))

    model.add(Flatten())

    # Optional Dropout for regularization
    model.add(Dropout(rate=hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)))

    model.add(Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
        ),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=1,
    directory='my_dir',
    project_name='dog_cat_tuning'
)

tuner.search(train_generator, epochs=10, validation_data=test_generator)

Trial 5 Complete [00h 56m 53s]
val_accuracy: 0.8600000143051147

Best val_accuracy So Far: 0.8600000143051147
Total elapsed time: 05h 05m 34s


# Try the model with new images

In [55]:
best_model = tuner.get_best_models(num_models=1)[0]
best_model.save('dog_cat_classifier.keras')

model = load_model('dog_cat_classifier.keras')

img_path = 'aa.jpg'
img = image.load_img(img_path, target_size=(150, 150))
img_array = image.img_to_array(img) / 255.0  # normalize same as training
img_array = np.expand_dims(img_array, axis=0)  # add batch dimension

prediction = model.predict(img_array)[0][0]  # sigmoid output

if prediction > 0.5:
    print("It's a dog 🐶")
else:
    print("It's a cat 🐱")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
It's a cat 🐱
